# MARO  Supply Chain Extensibility Example

## Prerequisite

In [2]:
import os
from typing import Dict

In [2]:
sc_root = "/home/jinyu/maro/maro/simulator/scenarios/supply_chain"

In [3]:
def bold(text: str):
    return f"\033[1m{text}\033[0m"

def dirTree(dir):
    all_files = os.listdir(dir)
    dirs = [
        bold(name) for name in all_files
        if name not in ["__pycache__"] and os.path.isdir(os.path.join(dir, name))
        and ("SCI_" not in name or name == "SCI_1.1") 
    ]
    files = [
        name for name in all_files
        if name not in ["__init__.py"] and os.path.isfile(os.path.join(dir, name))
    ]
    print(f"|____{bold(os.path.split(dir)[1])}")
    for filename in dirs + files:
        print(f"  |____{filename}")

## File Topology

Supply Chain simulator folder

In [4]:
dirTree(sc_root)

|____supply_chain
  |____datamodels
  |____facilities
  |____topologies
  |____units
  |____order.py
  |____business_engine.py
  |____parser.py
  |____world.py
  |____objects.py
  |____frame_builder.py
  |____sku_dynamics_sampler.py
  |____actions.py


### facilities

New types of facilities could be added under folder **facilities**:
- `Supplier`: Usually used to simulate the facility providing products to downstream facilities. Corresponds to the **Vendor** & the **Plant**s in SCI scenario.
- `Warehouse`: Usually used to simulate the facility temporarily holding a lot of products. Corresponds to the **Storage Warehouse**s in SCI scenario.
- `Retailer`: Usually used to simulate the facility selling products to the end-point customers. Corresponds to the **Store**s in SCI scenario.

In [5]:
dirTree(os.path.join(sc_root, "facilities"))

|____facilities
  |____supplier.py
  |____warehouse.py
  |____facility.py
  |____retailer.py


### units

Functional units are defined under folder **units**:
- `Storage Unit`: For product storage. Handle the try_get_product() and try_add_product() actions, so as to maintain a correct inventory level. One storage unit correponds to one facility.  
- `Distribution Unit`: For receiving the orders from downstream facilities and in charge of product delivery to downstreams. One distribution unit correponds to one facility.
- `Porduct Unit`: The function combination for one specific product in one facility. One product unit corresponds to one facility, but one facility may has many product units.
  - `Consumer Unit`: The unit in charge of purchasing products from the upstream facilities. One product unit may have at most one consumer unit.
  - `Seller Unit`: The unit in charge of selling products to the end-point customers. One product unit may have at most one seller unit.
  - `Manufacture Unit`: The unit in charge of manufacturing products with given manufacture rate. One product unit may have at most one manufacture unit.

In [6]:
dirTree(os.path.join(sc_root, "units"))

|____units
  |____distribution.py
  |____extendunitbase.py
  |____product.py
  |____consumer.py
  |____seller.py
  |____storage.py
  |____manufacture.py
  |____unitbase.py


### datamodels

**Data models** are used to track the important status of **units** or **facilities** tick by tick.
It defines exactly what to save in the snapshot list, which means the whole historical data trajectory can be accessed at any time we need it.

In [7]:
dirTree(os.path.join(sc_root, "datamodels"))

|____datamodels
  |____distribution.py
  |____product.py
  |____consumer.py
  |____seller.py
  |____storage.py
  |____base.py
  |____manufacture.py
  |____facility.py
  |____extend.py


### topologies

The configuration files used to describe the topology network we want to simulate are saved in folder **Topologies**.

- The file *core.yml* defines the main **Facility** and **Unit** we are going to use and connect the **Unit** with its corresponding **Data model** together.

- Each topology folder corresponds to one topology/problem definition. The detailed configurations are written in the file *config.yml* as you can see the example below.  

In [8]:
dirTree(os.path.join(sc_root, "topologies"))

|____topologies
  |____plant
  |____random
  |____sample
  |____super_vendor
  |____SCI_1.1
  |____core.yml


In [9]:
dirTree(os.path.join(sc_root, "topologies/super_vendor"))

|____super_vendor
  |____config.yml


## Example I: Add a new Storage Unit to provide infinite stocks

If we want to new Unit whose function is not covered by current existing Units, we need to follow these steps:
1. Define the corresponding Unit.
2. Add it to the `core.yml`.
3. Modify or create a corresponding topology folder and the `config.yml` configuration file.
4. Use the new or updated topology config to run your experiments.

### Step 1: Define a new Super Storage Unit to support infinite product inventory.

Firstly, let's go through what a General Storage Unit may looks like:

In [10]:
class StorageUnit():
    _remaining_space: int
    _product_level: Dict[int, int]

    def try_add_products(self, product_quantities: Dict[int, int]) -> Dict[int, int]:
        space_requirement = sum(quantity for quantity in product_quantities.values())
        if space_requirement < self._remaining_space:
            return {}

        for sku_id, quantity in product_quantities.items():
            self._product_level[sku_id] += quantity
            
        return product_quantities

    def try_take_products(self, product_quantities: Dict[int, int]) -> bool:
        # Check if we can take all kinds of products?
        for sku_id, quantity in product_quantities.items():
            if self._product_level[sku_id] < quantity:
                return False

        # Take from storage.
        for sku_id, quantity in product_quantities.items():
            self._product_level[sku_id] -= quantity

        return True

    def take_available(self, sku_id: int, quantity: int) -> int:
        available = self._product_level[sku_id]
        self._product_level[sku_id] -= min(available, quantity)
        return actual

Later, to support the infinite product inventory, we may need a new class like:

In [11]:
class SuperStorageUnit():
    def try_add_products(self, product_quantities: Dict[int, int]) -> Dict[int, int]:
        return product_quantities

    def try_take_products(self, product_quantities: Dict[int, int]) -> bool:
        return True

    def take_available(self, sku_id: int, quantity: int) -> int:
        return quantity

The `SuperStorageUnit` is simpler than the general `StorageUnit`, it just satisfy all the needs. As for the corresponding **Data Model**, there are nothing important to keep tracking for `SuperStorageUnit`, so we don't need to define new data model class for it.

### Step 2: Add the newly defined `SuperStorageUnit` to *core.yml* file.

Referring the orginal definition of general `StorageUnit`, we can add the definition like below:

```yaml
datamodels:  # No updates to this part since we do not add any new data model.
  ...

units:
  modules:
    - path: "maro.simulator.scenarios.supply_chain.units"
      definitions:
        StorageUnit:  # The definition of the General Storage Unit.
          class: "StorageUnit"
          datamodel: "StorageDataModel"
        SuperStorageUnit:  # The definition of the newly added Super Storage Unit.
          class: "SuperStorageUnit"  # The class name of the newly added class.
          datamodel: null  # Do not need any data model, so do not connect any data model to it.
        ...

facilities:  # No updates to this part since we do not add any new facility.
  ...
```

### Step 3: Update the configuration in *config.yml* file.

As you can see the details in *plant/config.yml*, we define and use the *VendorFacility* as below:

```yaml
facility_definitions:
  VendorFacility:  # The definition of VendorFacility in topology plant.
    class: "SupplierFacility"
    datamodel: "FacilityDataModel"
    children:
      storage:
        class: "StorageUnit"  # The Storage Unit to use for VendorFacility
...
world:
  facilities:
    - name: "Vendor_001"
      definition_ref: "VendorFacility"  # Use the correspondind alias in facility_definitions part.
      children:
        storage:
          config:  # The detailed configuration of the used Genral StorageUnit.
            capacity: 80000
            unit_storage_cost: 0.1
...
```

To use the `SuperStorageUnit` instead in super_vendor topology, we modify this part in *super_vendor/config.yml* as below:

```yaml
facility_definitions:
  SuperVendor:  # The definition of SuperVendor in topology super_vendor. A different alias is used here.
    class: "SupplierFacility"
    datamodel: "FacilityDataModel"
    children:
      storage:
        class: "SuperStorageUnit"  # Use the SuperStorageUnit instead.
...
world:
  facilities:
    - name: "Vendor_001"
      definition_ref: "SuperVendor"  # Use the correspondind alias in facility_definitions part.
      children:
        storage:
          config:  # The detailed configuration of the used SuperStorageUnit.
            capacity: null  # null indicates infinite storage capacity.
            unit_storage_cost: 0  # Not need to count the storage cost for this kind of Storage Unit.
...
```

### Step 4: Create an env instance with the newly defined topology name as the parameter. 

Create env instance with the orginal *plant* topology would be:
```py
env = Env(scenario="supply_chain", topology="plant", start_tick=0, durations=100)
```

You can simply modify the given topology parameter to the name of the newly created topology if you put the topology config under folder *supply_chain/topologies*, take &super_vendor* as the example, it would be:
```py
env = Env(scenario="supply_chain", topology="super_vendor", start_tick=0, durations=100)
```

## Example II: Define and change the route network

The route network of a topology is defined in `world.topology` part in *config.yml* file. For each downstream facility's each product, we should specify the upstream facility and the correponding transportation cost & leading time info.

### A simple route network with 3 facilities, 2 products and 3 routes

For example, assume we have Facility **Warehouse_001**, **Store_001**, **Store_002**, and we want the route network like:

[![](https://mermaid.ink/img/pako:eNp1kMFqg0AQhl9lmVMKGtTQHhZSSNLecoqHXBbC4I5V0F1ZZwsh-u7ZJlYK0jkNH_838M8NCqsJJHw57CpxPCkjwuxWZ3RUWd_TJUnSlyfdr3K27i85zCSbyE7E8ftQWqsvqRQlyhJjdlgb8d3wNotEYXveJuu3QewXRrY0NrPxOojD_0bXoKGHkU5Gtk6CARG05FqsdSh5-_EVcEUtKZBh1VSib1iBMmOI-k4j06euQysIZ5ueIkDPNr-aAiQ7T7-hjxrDz9opNd4BVOtmcg)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNp1kMFqg0AQhl9lmVMKGtTQHhZSSNLecoqHXBbC4I5V0F1ZZwsh-u7ZJlYK0jkNH_838M8NCqsJJHw57CpxPCkjwuxWZ3RUWd_TJUnSlyfdr3K27i85zCSbyE7E8ftQWqsvqRQlyhJjdlgb8d3wNotEYXveJuu3QewXRrY0NrPxOojD_0bXoKGHkU5Gtk6CARG05FqsdSh5-_EVcEUtKZBh1VSib1iBMmOI-k4j06euQysIZ5ueIkDPNr-aAiQ7T7-hjxrDz9opNd4BVOtmcg)

We can define the part `world.topology` as below:

```yaml
world:
  ...
  topology:
    "Store_001":  # Store_001 as the downstream facility.
      "food_1":   # For product food_1.
        "Warehouse_001":  # Warehouse_001 as the upstream facility.
          "train":  # Only 1 kind of route/vehicle type.
            vlt: 2
            cost: 0.6
    "Store_002":  # Store_002 as the downstream facility.
      "food_2":   # For product food_2.
        "Warehouse_001":  # Warehouse_001 as the upstream facility
          "train":  # There are 2 kinds of routes/vehicle types. The first one is by train.
            vlt: 3
            cost: 0.5
          "air":    # The second one is by air.
            vlt: 1
            cost: 2.0
```

### Extend the route network to 5 routes

If we want to delivery *food_2* from both **Warehouse_001** and **Store_002** to **Store_001**, and build up a route network like:

[![](https://mermaid.ink/img/pako:eNqN0VFrwjAQB_CvEu5JoZW2wzECDrT6tqf5sJeAHM11LbRJSS_CsH53M9eVwVDMU_jz_4Vwd4LCagIJnw67Sry9KyPCWc8-0FFlfU-HJEnnP-lmtmfr_ib5lGRjshZx_DqU1upDKkWJssSYHdZGHBteZZEobM-rZPE8iM0_kd0TLw-Kp0ksB5HfFl2Dhq4iHUW2SCaRPyTS668ggpZci7UOgzx9ewVcUUsKZLhqKtE3rECZc6j6TiPTTtdhchCebXqKAD3b_ZcpQLLz9Fva1hj20o6t8wVi6oRm)](https://mermaid-js.github.io/mermaid-live-editor/edit#pako:eNqN0VFrwjAQB_CvEu5JoZW2wzECDrT6tqf5sJeAHM11LbRJSS_CsH53M9eVwVDMU_jz_4Vwd4LCagIJnw67Sry9KyPCWc8-0FFlfU-HJEnnP-lmtmfr_ib5lGRjshZx_DqU1upDKkWJssSYHdZGHBteZZEobM-rZPE8iM0_kd0TLw-Kp0ksB5HfFl2Dhq4iHUW2SCaRPyTS668ggpZci7UOgzx9ewVcUUsKZLhqKtE3rECZc6j6TiPTTtdhchCebXqKAD3b_ZcpQLLz9Fva1hj20o6t8wVi6oRm)

We can update the `world.topology` part to:

```yaml
world:
  ...
  topology:
    "Store_001":
      "food_1":
        "Warehouse_001":
          "train":
            vlt: 2
            cost: 0.6
      "food_2":  # Newly added product.
        "Warehouse_001":  # Store_001 can purchase food_2 from Warehouse_001 by train.
          "train":
            vlt: 2
            cost: 0.8
        "Store_002":  # Store_001 can also purchase food_2 from Store_002 by air.
          "air":
            vlt: 1
            cost: 1.8
    "Store_002":
      "food_2":
        "Warehouse_001":
          "train":
            vlt: 3
            cost: 0.5
          "air":
            vlt: 1
            cost: 2.0
```